In [69]:
from bs4 import BeautifulSoup
import glob
import pypandoc
from markdownify import markdownify as md



In [ ]:


glob.glob("../ethical/mle/*")

newsletter_files = glob.glob("../ethical/mle/*")

In [107]:

def fetch_clean_newsletter(file):

    f = open(file)
    lines = f.readlines()
    
    html = BeautifulSoup("\n".join(lines), "html.parser")
    main_content = html.find("table").parent


    for tag in main_content():
            for attribute in ["class", "id", "name", "style", "data-darkreader-inline-color", "align"]:
                    del tag[attribute] 
    
    invalid_tags = ["table", "tbody", "thead", "tr", "td", "th", "tfoot", "b", "div", "img", "span", "strong"]
    for tag in invalid_tags:
            for match in main_content.findAll(tag):
                    match.replaceWithChildren()

    table_str = str(main_content)

    for t in invalid_tags:
        table_str = table_str.replace(f"</{t}>", "")
    
    table_str = table_str.replace("<a href=\"#\"></a>", "")
    table_str = table_str.replace("<a href=\'#\'></a>", "")

    table_md_articles = md(table_str)

    table_md_articles = table_md_articles.split("\nUpcoming MLOps Events\n")[0]
    table_md_articles = table_md_articles.split("MLOps = ")[0]
    table_md_articles = table_md_articles.split("MLJobs = ")[0]
    table_md_articles = table_md_articles.split(" OSS: ")[0]

    html_table = pypandoc.convert_text(table_md_articles, "html", format="md")
    
    return html_table

In [108]:
newsletters = []

for i in range(len(newsletter_files)):
    newsletter_file = f"../ethical/mle/{i+1}.html"
    print(newsletter_file)
    newsletter = fetch_clean_newsletter(newsletter_file)
    newsletters.append(newsletter)


../ethical/mle/1.html
../ethical/mle/2.html
../ethical/mle/3.html
../ethical/mle/4.html
../ethical/mle/5.html
../ethical/mle/6.html
../ethical/mle/7.html
../ethical/mle/8.html
../ethical/mle/9.html
../ethical/mle/10.html
../ethical/mle/11.html
../ethical/mle/12.html
../ethical/mle/13.html
../ethical/mle/14.html
../ethical/mle/15.html
../ethical/mle/16.html
../ethical/mle/17.html
../ethical/mle/18.html
../ethical/mle/19.html
../ethical/mle/20.html
../ethical/mle/21.html
../ethical/mle/22.html
../ethical/mle/23.html
../ethical/mle/24.html
../ethical/mle/25.html
../ethical/mle/26.html
../ethical/mle/27.html
../ethical/mle/28.html
../ethical/mle/29.html
../ethical/mle/30.html
../ethical/mle/31.html
../ethical/mle/32.html
../ethical/mle/33.html
../ethical/mle/34.html
../ethical/mle/35.html
../ethical/mle/36.html
../ethical/mle/37.html
../ethical/mle/38.html
../ethical/mle/39.html
../ethical/mle/40.html
../ethical/mle/41.html
../ethical/mle/42.html
../ethical/mle/43.html
../ethical/mle/44.ht

In [110]:
print(newsletters[2])

<p>THE ML ENGINEER</p>
<p>Issue #3</p>
<p> </p>
<p>  This week in <a href="https://ethical.institute/mle/3.html">Issue
#3:</a> Extreme ML with <a
href="https://dzone.com/articles/machine-learning-infrastructure-for-extreme-scale">Apache
Kafka</a>, programming <a
href="https://medium.com/@BonsaiAI/what-do-we-want-from-explainable-ai-5ed12cb36c07">explainable
ML</a>, broad overview of <a
href="https://towardsdatascience.com/how-to-do-everything-in-computer-vision-2b442c469928">computer
vision</a>, going <a
href="https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/">beyond
accuracy</a>, deep <a
href="https://people.cs.umass.edu/~hzjiang/projects/superslomo/">slow-mo
generators</a>, guidelines for <a
href="https://ec.europa.eu/digital-single-market/en/news/draft-ethics-guidelines-trustworthy-ai">trustworthy
AI</a>, explainability <a
href="https://github.com/EthicalML/awesome-machine-learning-operations">open
source libraries</a> and more! 